In [81]:
import numpy as np
import scipy as sp
import scipy.integrate as integrate


R = 8.3145 # J/(mol K)
Tpad = 298.15
Comp = {
    "H2" : {
        "A": 3.249,
        "B": 0.422e3,
        "C": 0,
        "D": 0.083e-5
    },
    "CO" : {
        "A": 3.376,
        "B": 0.557e3,
        "C": 0,
        "D": -0.031e-5
    },
    "CO2" : {
        "A": 5.457,
        "B": 1.045e3,
        "C": 0,
        "D": -1.157e-5
    },
    "N2" : {
        "A": 3.280,
        "B": 0.593e3,
        "C": 0,
        "D": 0.040e-5
    },
    "CH3OH": {
        "omega" : 0.565,
        "Pc" : 80.97,
        "Tc" : 512.64,
        "Tb" : 337.69,
        "A": 2.211,
        "B": 12.216e3,
        "C": -3.450e6,
        "D": 0
    },
    "H2O" : {
        "omega" : 0.344, # Prausnitz
        "Pc" : 220.64, #Bar
        "Tb" : 373.15, #K
        "Tc" : 647.14, #K
        "A": 3.470, # V.Ness - |
        "B": 1.450e3,
        "C": 0,
        "D": 0.121e-5
    }
}

def toK(c):
    return (c + 273.15)

def Cp_ig(T, A, B, C, D):
    return R * (A + (B * T) + (C * (T ** 2)) + (D * (T ** (-2))))

def Cp_L(T, Tc, omega):
    return R * (1.45 + (0.45/(1-(T / Tc)) + 0.25 * omega * (17.11 + (25.2 * (1 - (T / Tc))**(1/3)) / (T / Tc) + (1.742)/(1 - (T / Tc)) ) ) )

def Hres(P, T, Tc, a, alpha, beta, gamma, omega):
    
    pass

In [82]:
Hig_h2o = integrate.quad(Cp_ig, 273.15, toK(100), args=
(Comp["H2O"]["A"],
Comp["H2O"]["B"],
Comp["H2O"]["C"],
Comp["H2O"]["D"],
))

HL_h2o = integrate.quad(Cp_L, 273.15, Comp["H2O"]["Tb"], args=
(
    Comp["H2O"]["Tc"],
    Comp["H2O"]["omega"]
)
)
print(f'int HL = {HL_h2o}\n')

Hig_h2o = Hig_h2o[0] + HL_h2o[0]
print(f'{Hig_h2o/1000} kJ/mol')

int HL = (6326.884488066894, 7.024252832795612e-11)

389599.659890988 kJ/mol


In [83]:
import chemicals.heat_capacity as hc


water_low_gas_coeffs = [30.09200, 6.832514/1e3, 6.793435/1e6, -2.534480/1e9, 0.082139*1e6]
gi = hc.Shomate_integral(toK(100), *water_low_gas_coeffs)

print(integrate.quad(hc.Rowlinson_Bondi, 298.15, toK(100), (Comp["H2O"]["Tc"], Comp["H2O"]["omega"], gi))[0] / 1000)

873.8873092746368
